# Fiat & Shamir 1986 Identification scheme

Full text available in [Fiat & Shamir 1986 How to prove yourself](https://link.springer.com/content/pdf/10.1007/3-540-47721-7_12.pdf).

FS 1986 shows how to use Zero knowledge techniques for identification purposes. Their method allows an entity to provide an identity proof or authenticate a message without shared or public keys. It is secure against known or chosen message attacks. The scheme is based on the difficulty of extracting [modular square roots](https://www.rieselprime.de/ziki/Modular_square_root#:~:text=A%20modular%20square%20root%20of,2%20%E2%89%A1%20a%20(%20mod%20m%20)&text=Otherwise%2C%20the%20number%20of%20square,quadratic%20residue%20modulo%20or%20not.) when the factors of $n$ are unknown (essential if we want to protect the prover against a malicious verifier).

FS distinguish between three levels of protection:

1. Authentication schemes: Only A can prove to B that he is A.
2. Identification schemes: A can prove to B that he is A, but B cannot prove to someone else that he is A.
3. Signature schemes: A can prove to B that he is A, but B cannot prove even to himself that he is A.

The distinction between 2. and 3. above is mainly that an identification scheme would allow B to create a credible transcript of an imaginary communication by carefully choosing both the questions and the answers. In contrast, a signature scheme require a real communication to generate a credible transcript. In real time scenarios the two schemes are equivalent.

The scheme assumes a TTP that can verify a physical identity and link this identity to a digital identity. The scheme itself does not require TTP involvement.

## The scheme

1. A TTP picks a public modulus $n=pq$, where $(p,q)\in\mathbb{P}$, and a pseudo random function $f$ that maps strings to the range $[0,n)$
2. The TTP prepares a string $I$ with all the relevant and verified identifiable information about the user (name, address, ID number, physical traits etc.)
3. The TTP computes the values $v_j=f(I,j)$ for small values of $j$.
4. The TTP picks $k$ distinct values of $j$ for which $v_j$ is a quadratic residue $\mod n$ and computes the smallest square root $s_j$ of $v_j^{-1} \mod n$
5. Issue an object (e.g., smart card) that contains $I$ the $k$ values of $s_j$ and their indices. 

A fully decentralized version would require users to pick their own $n$ and publish it in a public key directory.

Verification only requires knowledge of $(n,f)$. The verifier issues a challenge and the prover (e.g., the smart card) generates a ZKP of $\mathbf{s}=\{s_0,s_1,\ldots,s_k\}$ in the following way:

1. Peggy sends $I$ to Victor.
2. Victor generates $v_j=f(I,j)$ for $j=0,\ldots,k$
3. Victor decides on the number of tests, $t$ to perform and repeats:
 1. Peggy picks a random $r\in[0,n)$ and sends $x_i = r_i^2 \mod n$ to Victor. Alternatively, Peggy can send a part of the hash, e.g., $H(x_i)[:128]$
 2. Victor sends a random binary vector $\{e_{i0},\ldots,e_{ik}\}$ to Peggy
 3. Peggy sends $y_i = r_i \prod_{e_{ij}=0}^{k} s_{j} \mod{n}$
 4. Victor checks that $x_i = y_i^2 \prod_{e_{ij}=0}^k v_j \mod{n}$. Alternatively, Victor checks the hash of the right hand side.
 
The verifier accepts the proof of identity if all $t$ checks are successful.

## Notes on square root modulo $N$ where $N = pq$ and where $(p,q)\in \mathbb{P}$

Resources on how to find sqrt of certain primes: 

* https://math.stackexchange.com/questions/633160/modular-arithmetic-find-the-square-root/633174#:~:text=Again%2C%20if%20a%20square%20root,%E2%8B%852%3D4%20square%20roots.
* https://crypto.stackexchange.com/questions/29949/how-to-use-crt-to-compute-4-square-roots-while-decryption-in-rabin-cryptosystem
* https://www.cs.purdue.edu/homes/ssw/cs655/week5.pdf
* https://math.stackexchange.com/questions/2132422/calculate-square-roots-using-crt
* https://math.stackexchange.com/questions/2856758/chinese-remainder-theorem-four-square-roots-of-1-modulo-n
* https://xuan-li.github.io/post/chinese-remainder/
* https://en.wikipedia.org/wiki/Tonelli%E2%80%93Shanks_algorithm
* https://en.wikipedia.org/wiki/Cipolla%27s_algorithm
* https://math.stackexchange.com/questions/95443/how-to-compute-modular-square-roots-when-modulus-is-non-prime

And a collection of known algos here: https://minds.wisconsin.edu/bitstream/handle/1793/11024/file_1.pdf?sequence=1&isAllowed=y

To make things simple, I will use the libnum library below. If the prime factors are known, it is efficient to find the square root mod n.

In [1]:
from Crypto.Util import number
from Crypto.Hash import SHA256
import libnum

In [2]:
def v_j(id_string, n=10):
    """
    Takes as input an id string
    Outputs n values for vj
    """
    v = []
    for i in range(n):
        a = SHA256.new()
        a.update(id_string.encode('utf-8') + str(i).encode('utf-8'))
        v.append(int(a.hexdigest(), base=16))
    return v

def s_j(v, p, q):
    """
    Takes as input a list of quadratic residues
    Outputs the root of the inverse is such a root exists
    """
    sj = []
    for i in v:
        temp = pow(i,-1,p*q)
        if root_test(temp,p,q):
            sj.append(next(libnum.sqrtmod(temp, {p: 1, q: 1})))
    return sj

def quad_res_index(v,p,q):
    """
    Takes as input a list of integers
    Returns a list of indexes for those that are quad residues
    """
    quadratic_residues_index = [a for a,x in enumerate([libnum.has_sqrtmod(v[i], {p: 1, q: 1}) for i in range(len(v))]) if x == True]
    return quadratic_residues_index

def setup(bitlength=256):
    assert bitlength >= 256, 'Paper recommends bitlenth above 512 for n'
    n = 0
    while n.bit_length() < 512:
        p = number.getPrime(bitlength)
        q = number.getPrime(bitlength)
        n = p*q
    return p,q,n

In [3]:
# TTP's or Peggy's parts
p, q, n = setup()    
I = 'Peggy, Homestreet 99, 123456, brown eyes'

# Peggy's parts
j = 2 ** 8 - 1 # Just pick an arbitrary small number of values for j
v = v_j(I, j) # These are public

quadratic_residues_index = quad_res_index(v,p,q)
valid_vj_dict = {} # These are the values for v_j that are quad residues mod n
for i in quadratic_residues_index:
    valid_vj_dict[i] = v[i]
    
valid_vj_dict_inv = {}
for i in quadratic_residues_index:
    valid_vj_dict_inv[i] = pow(v[i], -1, p*q)

assert [libnum.has_sqrtmod(valid_vj_dict_inv[i], {q:1,p:1}) for i in quadratic_residues_index].count(False) == 0, 'Check that the inverses have mod sqrt'

sj_dict = {}
for i in quadratic_residues_index:
    sj_dict[i] = next(libnum.sqrtmod(valid_vj_dict_inv[i], {q:1,p:1}))
    
assert [pow(sj_dict[i],2,p*q) == valid_vj_dict_inv[i] for i in quadratic_residues_index].count(False) == 0, 'At least one sj value does not square to the right vj value'

In [4]:
print(f'The smartcard will contain "{I}" and the following sj values with indexes:')
sj_dict

The smartcard will contain "Peggy, Homestreet 99, 123456, brown eyes" and the following sj values with indexes:


{2: 3934726974507871180213407881251875284212621965672557900068629587690937301644043346837766520507766911112296092664578313105124239610703410773678020567298427,
 3: 821069622005275389755296304303233173961912027051112509290083440909040091604989685754356339452871988622116511273740873766194805734125936607768441574461735,
 5: 29682653942977574225220984052397836092726268477459433488927522569833587346436052397601021394392371972097554993128125706213630390120520422071953852559641,
 8: 310708665028438259242563617050945759865415719128450406299537976694751575232673701842330142499313002112991110839148679565729256780726024715922713788303837,
 10: 1002153312664990960365963321902342985587246708257253440636003652184045183118960578540392680085390462991635501142166538592710600101770487749211073333711528,
 19: 1942913760551845429975146727896216736772381746906350736022467202553463693478594829983716765173038416644647908758171754580974281660051197169069340902407141,
 23: 5361718976261636148197167963120881935

In [5]:
# Peggy shows I to Victor, who now does the following
test_indexes = libnum.random.sample(quadratic_residues_index, k=libnum.random.randrange(1,len(sj_dict))) # Victor randomly sammples a subset of v where elements are all quadratic residues mod n
test_indexes.sort()
test_vj = {}
for i in test_indexes:
    temp = None
    temp = SHA256.new()
    temp.update(I.encode() + str(i).encode())
    test_vj[i] = int(temp.hexdigest(), base=16)
    
# Peggy now picks a random r in [0,n) and sends r^2 mod n to Victor
r_i = libnum.random.randint(0,n-1)
x_i = pow(r_i,2,n)

# Victor generates a binary vector to Peggy
test_binary_vector = [0]*len(test_vj)
while test_binary_vector.count(1) == 0:
    test_binary_vector = libnum.random.choices([0,1], k=len(test_vj))
test_vj_list = [test_vj[i] for i in [a for a in test_vj]] # These are the vj values that correspond to the subset Victor picked

# Peggy now computes
test_sj_list = [sj_dict[i] for i in [a for a in test_vj]] # These are the sj values that correspond to the subset Victor picked
y_i = r_i
for i in [test_sj_list[i] for i in [x for x,a in enumerate(test_binary_vector) if a == 1]]:
    y_i *= i
    y_i = y_i % n
    
# Victor gets y_i and computes
test_x_i = pow(y_i,2,n)
for i in [test_vj_list[i] for i in [x for x,a in enumerate(test_binary_vector) if a == 1]]:
    test_x_i *= i
    test_x_i = test_x_i % n

print('Victor performs check 3D above and it is valid:',test_x_i == x_i)
print('Likelihood of Peggy random guessing is:', 1 / 2 ** len(test_indexes))

Victor performs check 3D above and it is valid: True
Likelihood of Peggy random guessing is: 1.1641532182693481e-10
